In [2]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from mtcnn import MTCNN  # Thư viện phát hiện khuôn mặt

In [3]:
# Định nghĩa kích thước hình ảnh
IMG_SIZE = (128, 128)  # Kích thước đầu vào cho mô hình

# Load mô hình đã huấn luyện
model = load_model("face_recognition_model.h5")

In [ ]:
# Danh sách các nhãn

class_names = [
    'DoMixi', 
    'HariWon', 
    'HieuThuHai', 
    'HoNgocHa', 
    'Misthy', 
    'PhuongAnhDao', 
    'SonTungMTP', 
    'ThuyTien', 
    'TranThanh', 
    'karik'
]


In [20]:
# Ngưỡng xác định "Unknown" (ví dụ: nếu độ tin cậy < 50%)
CONFIDENCE_THRESHOLD = 50.0

# Đường dẫn thư mục chứa ảnh
image_folder = r"C:\DL\NDKM\img_test"

# Khởi tạo bộ phát hiện khuôn mặt MTCNN
detector = MTCNN()

In [21]:
# Duyệt qua tất cả ảnh trong thư mục
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    
    # Đọc ảnh
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc ảnh: {image_name}. Bỏ qua.")
        continue

    # Phát hiện khuôn mặt trên ảnh
    results = detector.detect_faces(image)

    for result in results:
        # Lấy vị trí khuôn mặt
        x, y, width, height = result['box']
        x, y = max(0, x), max(0, y)  # Đảm bảo không âm
        face = image[y:y+height, x:x+width]  # Crop khuôn mặt

        # Resize khuôn mặt về kích thước chuẩn
        try:
            resized_face = cv2.resize(face, IMG_SIZE)
        except Exception as e:
            print(f"Resize error on {image_name}: {e}")
            continue

        # Chuẩn hóa khuôn mặt
        normalized_face = resized_face / 255.0
        reshaped_face = np.expand_dims(normalized_face, axis=0)

        # Dự đoán nhãn
        predictions = model.predict(reshaped_face, verbose=0)
        confidence = np.max(predictions) * 100  # Tính toán độ tin cậy
        if confidence < CONFIDENCE_THRESHOLD:
            predicted_label = "Unknown"
        else:
            predicted_label = class_names[np.argmax(predictions)]

        # Vẽ khung quanh khuôn mặt
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(image, f'{predicted_label} ({confidence:.2f}%)', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Hiển thị ảnh đã xử lý
    cv2.imshow(f"Processed Image - {image_name}", image)

    # Nhấn phím bất kỳ để chuyển sang ảnh tiếp theo, nhấn 'q' để thoát
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

# Đóng tất cả cửa sổ
cv2.destroyAllWindows()